In [4]:
import pandas as pd
from collections import Counter
from itertools import permutations 
from itertools import combinations_with_replacement 

vocab = set()

def readFile(filename):
    f=open(filename, "r")
    if f.mode == 'r':
        contents =f.read()
    return contents

def writeFile(filename, data):
    file1 = open(filename,"w+")
    file1.writelines(data) 
    file1.close() 

In [5]:
def calculateFreq(data):
    print("Preprocessing data")
    data="ST/S "+ data
    data = data.replace('\n'," EN/E ST/S ")[:-5]
    ogData = data
    #deleting the tags
    tags = ['/A','/C','/D','/M','/N','/O','/P','/R','/V','/W','/E','/S']
    for i in tags:
        data = data.replace(i,'')
   
    #claculating frequency    
    freq = Counter(data.split(' '))
    
    #generating final string
    unknownSet = set()
    for key,values in freq.items():
        if values < 2:
            unknownSet.add(key)
        else: vocab.add(key)
    
    result = ''    
    for i in ogData.split():   
        words = i.split('/')
        if words[0] in unknownSet or words[0] not in vocab:
            result+='UNK/'+words[1]+' '
        else: result += words[0]+"/"+words[1]+' '
    return result


def transProba(data, alpha):
    print("Calculating transitional probabilities")
    tags = ['A', 'C', 'D', 'M', 'N', 'O', 'P', 'R', 'V', 'W', 'E', 'S']
    perm = permutations(tags,2) 
    comb = combinations_with_replacement(tags,2) 
    dictComb = {}
    dict = {}
    
    #Generating dictionaries
    for i in perm:
        if i[0] != 'S' and i[1] != 'E':
            dictComb[i] = 0
    for i in comb:
        if i[0] != 'S' and i[1] != 'E':
            dictComb[i] = 0
    for i in tags:
        dict[i] = 0
    dict['S'] = 1 
    data = data.split()
    i=1
    
    #generating counts for (y,y') and (y')
    while i < len(data):
        try:
            words = data[i].split('/')
            prevWords = data[i-1].split('/')
            if words[1] != 'S':
                dictComb[(words[1], prevWords[1])]+=1
            dict[words[1]]+=1
            i+=1
        except:
            dict[words[1]]+=1
            i+=1
            
    #calculating transitional probabilities
    result = {}
    for key in dictComb:
        try:
            result[key] = (dictComb[key]+alpha) / (dict[key[1]]+(alpha*(len(tags)+1-2)))
        except:
             result[key] = 0         
    return result 

def emmiProba(data,beta):
    print("Calculating emmision probabilities")
    dictComb = {}
    dict = {}
    for i in data.split():
        word = i.split('/')
        
        key = (word[0],word[1])
        
        if key in dictComb: dictComb[key]+=1
        else: dictComb[key] = 1
            
        if word[1] in dict: dict[word[1]]+=1
        else: dict[word[1]] = 1
    
    for key in dictComb:
        try:
            dictComb[key] = (dictComb[key]+beta) / (dict[key[1]]+(beta*(len(vocab))))
        except:
            dictComb[key] = 0
    return dictComb

In [6]:
# test = "In/C of/C aftermath/N of/C the/D stock/N market/N 's/P gut-wrenching/A 190-point/A drop/N on/C Oct./N 13/O ,/O Kidder/N ,/O Peabody/N &/C Co./N 's/P 1,400/O stockbrokers/N across/C the/D country/N began/V a/D telephone/N\n"
# writeFile("data/processedTrn.txt", calculateFreq(data)[:-5])

xdata = readFile("data/trn.pos")
data =  calculateFreq(data)
tp = transProba(data,1)
ep = emmiProba(data,1)

Preprocessing data
Calculating transitional probabilities
Calculating emmision probabilities


In [31]:
def viterbi(transProb, emissionProb, sentence, tags):
    dp = [[[1,None] if i == 0 else [0, None] for i in range(len(sentence.split()))] for _ in range(len(tags))]
    words = sentence.split()
    
    for word in range(1,len(dp[0])):
        if word == 1:
            tag_seq = ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S']
        else: tag_seq = tags
        for tag in range(len(dp)):
            temp_score = 0
            back_item = None
            for item in range(len(dp)):              
                try:
                    tran = transProb[(tags[tag], tag_seq[item])]
                    emis =  emissionProb[(words[word].split('/')[0], tags[tag])]                     
                    score = tran * emis * dp[item][word-1][0]
                except:
                    score = 0
                if temp_score < score:
                    back_item = item
                temp_score = max(temp_score, score)
            dp[tag][word][0] = temp_score
            dp[tag][word][1] = back_item
    return dp

def backtrack(dp,tags):
    mx = 0
    mx_id = 0
    seq = [None]
    for i in range(len(dp)):
        if mx < dp[i][-2][0]:
            mx = dp[i][-2][0]
            mx_id = dp[i][-2][1]
            seq[0] = i
            
    seq.append(mx_id)        
    ptr = mx_id        
    for i in range(len(dp[0])-3, 0, -1):
#         print(ptr,i, dp[ptr][i])
        ptr = dp[ptr][i][1]
        seq.append(ptr)
    pos = []
    for i in seq:
        pos.append(tags[i])
    return pos[-1::-1]
tags = ['A', 'C', 'D', 'M', 'N', 'O', 'P', 'R', 'V', 'W']


def processTest(data):
    print("Preprocessing data")
    data="ST/S "+ data
    data = data.replace('\n'," EN/E ST/S ")[:-5]
    ogData = data
    
    #deleting the tags
    tags = ['/A','/C','/D','/M','/N','/O','/P','/R','/V','/W','/E','/S']
    for i in tags:
        data = data.replace(i,'')
        
    #generating final string    
    result = ''    
    for i in ogData.split():   
        words = i.split('/')
        if words[0] not in vocab:
            result+='UNK/'+words[1]+' '
        else: result += words[0]+"/"+words[1]+' '
    return result

def extractTags(data):
    result = []
    for i in data.split():   
        words = i.split('/')
        result.append(words[1])
    return result

def calacc(gt,pred):
    correct = 0
    for i,j in zip(gt,pred):
        if i==j:
            correct+=1
    return correct,len(gt)

def pipeline(test_data):
    dp = []
    acc = []
    for i in test_data.split('/E'):
        try:
            sample = i + '/E'
            gt = extractTags(sample)
            dp = viterbi(tp, ep, sample, tags)
            ta = backtrack(dp,tags)
            acc.append(calacc(gt[1:-1], ta[1:]))
        except:
            print('')
    return acc, dp

- Convert the estimated probabilities from the previous step into log space and im-plement the Viterbi decoding algorithm as in Algorithm 11in [Eisenstein, 2018].  Report theaccuracy of your decoder on the dev setdev.pos
## Best accuracy is 0.9566347008709313 

In [38]:
dev = readFile("data/dev.pos")
dev_data = processTest(dev)
dev_acc, tst_dp = pipeline(dev_data)
correct, total = 0, 0
for i in dev_acc:
    total+=i[1]
    correct+=i[0]
print("Accuracy is: ",correct/total)

Preprocessing data



Accuracy is:  0.9566347008709313


- Run the model selected from the previous step on the test settst.pos, and reportthe accuracy
## Best accuracy is 0.9590364050774317

In [40]:
test = readFile("data/tst.pos")
test_data = processTest(test)
tst_acc, tst_dp = pipeline(test_data)

correct, total = 0, 0
for i in tst_acc:
    total+=i[1]
    correct+=i[0]

print("Accuracy is: ",correct/total)



Preprocessing data


Accuracy is:  0.9590364050774317


- Tune α and β to find the best accuracy on the dev set.  Report the best accuracynumber and the values of α and β.
## Best value for α and β are 1 and 1 respectively; giving accuracy of: 0.95 on both dev and tst set.